In [1]:
class FieldElement:
    def __init__(self, num, prime):
        if num >= prime or num < 0:
            raise ValueError("Number {0} is not in the range between 0 and {1} - 1".format(num, prime))
        self.num = num
        self.prime = prime
        
    def __eq__(self, other):
        if not other:
            return False
        elif self.num == other.num and self.prime == other.prime:
            return True
        else:
            return False
        
    def __ne__(self, other):
        if not other:
            return False
        elif self.num != other.num or self.prime != other.prime:
            return True
        else:
            return False
        
    def __repr__(self):
        return "FieldElement_{0}({1})".format(self.prime, self.num)
    
    def __add__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num + other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __sub__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __mul__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __rmul__(self, scalor):
        num = (self.num * scalor) % self.prime
        return self.__class__(num, self.prime)
    
    def __truediv__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num * pow(other.num, other.prime - 2, other.prime)) % self.prime
        return self.__class__(num, self.prime)
    
    def __pow__(self, exponent):
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)
    
class Point:
    def __init__(self, x, y, a, b):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        if self.x == None and self.y == None:
            return 
        if self.y ** 2 != self.x ** 3 + a * x + b:
            raise ValueError("({0}, {1}) is not on the curve.".format(x, y))
            
    def __repr__(self):
        if self.x is None and self.y is None:
            return "Point(Infinity)"
        elif isinstance(self.x, FieldElement):
            return "Point({0}, {1})_{2}_{3} FieldElement({4})".format(self.x.num, self.y.num, self.a.num, self.b.num, self.x.prime)
        else:
            return "Point({0}, {1})_{2}_{3}".format(self.x, self.y, self.a, self.b)
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.a == other.a and self.b == other.b
    
    def __ne__(self, other):
        return self.x != other.x or self.y != other.y or self.a != other.a or self.b != other.b
    
    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError("Point {0} and {1} are not on the same curve".format(self, other))
            
        if self.x is None:
            return other
        
        if other.x is None:
            return self
        
        if self.x == other.x and self.y != other.y:
            return self.__class__(None, None, self.a, self.b)
        
        if self.x != other.x:
            s = (other.y - self.y)/(other.x - self.x)
            x = s ** 2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__(x, y, self.a, self.b)
        
        if self == other:
            if self.y == 0:
                return self.__class__(None, None, self.a, self.b)
            else:
                s = (3 * self.x ** 2 + self.a) / (2 * self.y)
                x = s ** 2 - 2 * self.x
                y = s * (self.x - x) - self.y
                return self.__class__(x, y, self.a, self.b)
    
    def __rmul__(self, scalor):
        s = scalor
        current = self
        res = self.__class__(None, None, self.a, self.b)
        while s:
            if s & 1:
                res += current
            current += current 
            s >>= 1
        return res

class S256Field(FieldElement):
    def __init__(self, num, prime = None):
        super().__init__(num = num, prime = P)
        
class S256Point(Point):
    def __init__(self, x, y, a = None, b = None):
        a, b = S256Field(A), S256Field(B)
        if type(x) == int:
            super().__init__(S256Field(x), S256Field(y), a, b)
        else:
            super().__init__(x, y, a, b)
    
    def __repr__(self):
        if self.x is None and self.y is None:
            return "S256Point(Infinity)"
        else:
            return "S256Point({0}, {1})".format(self.x, self.y)
        
    def __rmul__(self, scalor):
        s = scalor % N
        return super().__rmul__(s)
    
### Define secp256k1 ###
A = 0
B = 7
P = 2**256 - 2**32 - 977
X = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
Y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [2]:
prime = 223
a = FieldElement(0, prime)
b = FieldElement(7, prime)
x1 = FieldElement(192, prime)
y1 = FieldElement(105, prime)
x2 = FieldElement(17, prime)
y2 = FieldElement(56, prime)
P1 = Point(x1, y1, a, b)
P2 = Point(x2, y2, a, b)
P1 + P2

Point(170, 142)_0_7 FieldElement(223)

In [3]:
prime = 223
a = FieldElement(0, prime)
b = FieldElement(7, prime)
x = FieldElement(47, prime)
y = FieldElement(71, prime)
P0 = Point(x, y, a, b)
for s in range(1, 224):
    print("s = {0}, res = {1}".format(s, s * P0))

s = 1, res = Point(47, 71)_0_7 FieldElement(223)
s = 2, res = Point(36, 111)_0_7 FieldElement(223)
s = 3, res = Point(15, 137)_0_7 FieldElement(223)
s = 4, res = Point(194, 51)_0_7 FieldElement(223)
s = 5, res = Point(126, 96)_0_7 FieldElement(223)
s = 6, res = Point(139, 137)_0_7 FieldElement(223)
s = 7, res = Point(92, 47)_0_7 FieldElement(223)
s = 8, res = Point(116, 55)_0_7 FieldElement(223)
s = 9, res = Point(69, 86)_0_7 FieldElement(223)
s = 10, res = Point(154, 150)_0_7 FieldElement(223)
s = 11, res = Point(154, 73)_0_7 FieldElement(223)
s = 12, res = Point(69, 137)_0_7 FieldElement(223)
s = 13, res = Point(116, 168)_0_7 FieldElement(223)
s = 14, res = Point(92, 176)_0_7 FieldElement(223)
s = 15, res = Point(139, 86)_0_7 FieldElement(223)
s = 16, res = Point(126, 127)_0_7 FieldElement(223)
s = 17, res = Point(194, 172)_0_7 FieldElement(223)
s = 18, res = Point(15, 86)_0_7 FieldElement(223)
s = 19, res = Point(36, 112)_0_7 FieldElement(223)
s = 20, res = Point(47, 152)_0_7 FieldE

In [4]:
21*P0

Point(Infinity)

In [5]:
### Define secp256k1 ###
a = 0
b = 7
p = 2**256 - 2**32 - 977
x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
G = Point(FieldElement(x, p), FieldElement(y, p), FieldElement(a, p), FieldElement(b, p))

In [6]:
print(G)

Point(55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)_0_7 FieldElement(115792089237316195423570985008687907853269984665640564039457584007908834671663)


In [7]:
n * G

Point(Infinity)

In [8]:
(n + 1)*G

Point(55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)_0_7 FieldElement(115792089237316195423570985008687907853269984665640564039457584007908834671663)

In [9]:
(n - 1)*G

Point(55066263022277343669578718895168534326250603453777594175500187360389116729240, 83121579216557378445487899878180864668798711284981320763518679672151497189239)_0_7 FieldElement(115792089237316195423570985008687907853269984665640564039457584007908834671663)

In [10]:
from random import SystemRandom
_sysrand = SystemRandom()
k = _sysrand.getrandbits(256)

In [11]:
k * G

Point(76324012147034086473860086255694544497458330842310039005609537054315584542954, 108300888880877735345624148604438007128126276161158402218173264528331587784258)_0_7 FieldElement(115792089237316195423570985008687907853269984665640564039457584007908834671663)

In [12]:
int(n).bit_length()

256

In [13]:
2**256

115792089237316195423570985008687907853269984665640564039457584007913129639936

In [14]:
n

115792089237316195423570985008687907852837564279074904382605163141518161494337

In [15]:
assert(n < 2 ** 256)

In [16]:
G = S256Point(X, Y)

In [17]:
N * G

S256Point(Infinity)

In [18]:
(N - 1)* G

S256Point(FieldElement_115792089237316195423570985008687907853269984665640564039457584007908834671663(55066263022277343669578718895168534326250603453777594175500187360389116729240), FieldElement_115792089237316195423570985008687907853269984665640564039457584007908834671663(83121579216557378445487899878180864668798711284981320763518679672151497189239))

In [19]:
assert(G == (N + 1)* G)

#### Reference: "Programming Bitcoin: Learn How to Program Bitcoin from Scratch", 1st Edition, 2019